[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Anutri03/ClothingGAN/blob/master/ClothingGAN_Demo.ipynb)
# Clothing GAN demo


<br>
Make sure runtime type is GPU

In [1]:
#@title Install dependencies (restart runtime after installing)
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
#!pip install ninja gradio fbpca boto3 requests==2.23.0 urllib3==1.25.11

<IPython.core.display.Javascript object>

In [2]:
#!git clone https://github.com/Anutri03/ClothingGAN.git
#%cd ClothingGAN/

In [3]:
#@title Install other dependencies
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"

<IPython.core.display.Javascript object>

[nltk_data] Downloading package wordnet to C:\Users\Anurag
[nltk_data]     Tripathi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
#@title Load Model
selected_model = 'lookbook'

# Load model
from IPython.utils import io
import torch
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN2',
  layer='style',
  output_class=selected_model,
  components=80,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model

comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


In [5]:
#@title Define functions
from ipywidgets import fixed

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def name_direction(sender):
  if not text.value:
    print('Please name the direction before saving')
    return
    
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = [num, start_layer.value, end_layer.value]
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

def mix_w(w1, w2, content, style):
    for i in range(0,5):
        w2[i] = w1[i] * (1 - content) + w2[i] * content

    for i in range(5, 16):
        w2[i] = w1[i] * (1 - style) + w2[i] * style
    
    return w2

def display_sample_pytorch(seed, truncation, directions, distances, scale, start, end, w=None, disp=True, save=None, noise_spec=None):
    # blockPrint()
    model.truncation = truncation
    if w is None:
        w = model.sample_latent(1, seed=seed).detach().cpu().numpy()
        w = [w]*model.get_max_latents() # one per layer
    else:
        w = [np.expand_dims(x, 0) for x in w]
    
    for l in range(start, end):
      for i in range(len(directions)):
        w[l] = w[l] + directions[i] * distances[i] * scale
    
    torch.cuda.empty_cache()
    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)
    
    
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')
    if disp:
      display(final_im)
    
    return final_im

def generate_mov(seed, truncation, direction_vec, scale, layers, n_frames, out_name = 'out', noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in log_progress(range(n_frames), name = "Generating frames"):
    print(f'\r{i} / {n_frames}', end='')
    w = model.sample_latent(1, seed=seed).cpu().numpy()

    model.truncation = truncation
    w = [w]*model.get_max_latents() # one per layer
    for l in layers:
      if l <= model.get_max_latents():
          w[l] = w[l] + direction_vec * offset * scale

    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8))
    imgs.append(out)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(img_as_ubyte(image))

In [6]:
import gradio as gr
import numpy as np
from PIL import Image

# Assume model, latent_dirs, mix_w, and display_sample_pytorch are already loaded/imported

def generate_image(seed1, seed2, content, style, truncation, 
                   c0, c1, c2, c3, c4, c5, c6, 
                   start_layer, end_layer, red, green, blue):
    seed1 = int(seed1)
    seed2 = int(seed2)

    scale = 1
    params = {'c0': c0,
              'c1': c1,
              'c2': c2,
              'c3': c3,
              'c4': c4,
              'c5': c5,
              'c6': c6}

    param_indexes = {'c0': 0,
                     'c1': 1,
                     'c2': 2,
                     'c3': 3,
                     'c4': 4,
                     'c5': 5,
                     'c6': 6}

    directions = []
    distances = []
    for k, v in params.items():
        directions.append(latent_dirs[param_indexes[k]])
        distances.append(v)

    w1 = model.sample_latent(1, seed=seed1).detach().cpu().numpy()
    w1 = [w1] * model.get_max_latents()
    im1 = model.sample_np(w1)

    w2 = model.sample_latent(1, seed=seed2).detach().cpu().numpy()
    w2 = [w2] * model.get_max_latents()
    im2 = model.sample_np(w2)
    combined_im = np.concatenate([im1, im2], axis=1)
    input_im = Image.fromarray((combined_im * 255).astype(np.uint8))

    mixed_w = mix_w(w1, w2, content, style)
    output_im = display_sample_pytorch(seed1, truncation, directions, distances, scale, int(start_layer), int(end_layer), w=mixed_w, disp=False)

    # Apply color transformation
    output_array = np.array(output_im).astype(float) / 255.0

    # Create a mask to apply color change only to specific areas
    mask = (np.mean(output_array, axis=2) > 0.1) & (np.mean(output_array, axis=2) < 0.9)

    for c in range(3):
        if c == 0:
            output_array[:, :, c] = np.where(mask, output_array[:, :, c] * (1 + red / 100), output_array[:, :, c])
        elif c == 1:
            output_array[:, :, c] = np.where(mask, output_array[:, :, c] * (1 + green / 100), output_array[:, :, c])
        else:
            output_array[:, :, c] = np.where(mask, output_array[:, :, c] * (1 + blue / 100), output_array[:, :, c])

    output_array = np.clip(output_array, 0, 1)
    output_array = (output_array * 255).astype(np.uint8)
    output_im = Image.fromarray(output_array)

    return input_im, output_im

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## ClothingGAN")

    with gr.Row():
        with gr.Column():
            seed1 = gr.Number(label="Seed 1", value=1)
            seed2 = gr.Number(label="Seed 2", value=2)
            content = gr.Slider(0, 1, value=0.5, label="Content Mixing")
            style = gr.Slider(0, 1, value=0.5, label="Style Mixing")
            truncation = gr.Slider(0, 1, value=0.7, label="Truncation")
            start_layer = gr.Number(label="Start Layer", value=0)
            end_layer = gr.Number(label="End Layer", value=14)

            gr.Markdown("### Style directions")
            slider_min_val = -20
            slider_max_val = 20

            c0 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Sleeve & Size")
            c1 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Dress - Jacket")
            c2 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Female Coat")
            c3 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Coat")
            c4 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Graphics")
            c5 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Dark")
            c6 = gr.Slider(slider_min_val, slider_max_val, step=1, value=0, label="Less Cleavage")

            gr.Markdown("### Color Adjustment")
            red = gr.Slider(-100, 100, step=1, value=0, label="Red % Change")
            green = gr.Slider(-100, 100, step=1, value=0, label="Green % Change")
            blue = gr.Slider(-100, 100, step=1, value=0, label="Blue % Change")

        with gr.Column():
            input_image = gr.Image(label="Input Mixed Image")
            output_image = gr.Image(label="Output Styled Image")

    inputs = [seed1, seed2, content, style, truncation,
              c0, c1, c2, c3, c4, c5, c6,
              start_layer, end_layer, red, green, blue]

    # Instead of a button, trigger on change of any input
    for inp in inputs:
        inp.change(fn=generate_image, inputs=inputs, outputs=[input_image, output_image])

demo.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [7]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())


11.8
True
NVIDIA GeForce RTX 3050 Laptop GPU


In [8]:
import pycuda.driver as drv
drv.init()
print(drv.Device(0).name())


NVIDIA GeForce RTX 3050 Laptop GPU


In [9]:
import pycuda.driver as cuda
import pycuda.autoinit
print(cuda.Device(0).name())


NVIDIA GeForce RTX 3050 Laptop GPU


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Anutri03/ClothingGAN/blob/master/ClothingGAN_Demo.ipynb)
# Clothing GAN demo


<br>
Make sure runtime type is GPU

In [10]:
import pycuda.driver as drv
drv.get_version()


(11, 8, 0)